# Generate initial conditions file for high resolution experiment
We want to initialize the high res run with the solution of the 10km rum. Therefore we have to map 2D and 3D velocities, SSH, salinity and temperature fields on the new grid.
## Preparation of ini file
Load high and low resolution grid, low resolution history and low resolution ini file. 

In [119]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt

hr_grd_path = os.path.join(os.pardir,'data','preprocessing','interim','waom5_MinDepth20m_rx10.3_grd.nc')
hr_grd = xr.open_dataset(hr_grd_path)

lr_grd_path = os.path.join(os.pardir,'data','preprocessing','interim','waom10_MinDepth20m_rx10.3_grd.nc')
lr_grd = xr.open_dataset(lr_grd_path)

lr_rst_path =  os.path.join(os.pardir,'data','raw','waom10_full_forcing','ocean_rst.nc')
lr_rst = xr.open_dataset(lr_rst_path).isel(ocean_time=0,two=-1,three=-1)

lr_ini_path = os.path.join(os.pardir,'data','preprocessing','processed','waom10_ini.nc')
lr_ini = xr.open_dataset(lr_ini_path)

Exception ignored in: <bound method netcdf_file.close of <scipy.io.netcdf.netcdf_file object at 0x7f4ab727ad68>>
Traceback (most recent call last):
  File "/home/ubuntu/bigStick/anaconda3/lib/python3.6/site-packages/scipy/io/netcdf.py", line 279, in close
    if not self.fp.closed:
AttributeError: 'netcdf_file' object has no attribute 'fp'
Exception ignored in: <bound method netcdf_file.close of <scipy.io.netcdf.netcdf_file object at 0x7f4ab73b4048>>
Traceback (most recent call last):
  File "/home/ubuntu/bigStick/anaconda3/lib/python3.6/site-packages/scipy/io/netcdf.py", line 279, in close
    if not self.fp.closed:
AttributeError: 'netcdf_file' object has no attribute 'fp'


Prepare high resolution ini file by dropping all horizontal grid dependent variables from the low resolution ini file. 

In [120]:
hr_ini = lr_ini.drop(['u','v','ubar','vbar','salt','temp','zeta','ocean_time'])
hr_ini

<xarray.Dataset>
Dimensions:  (one: 1, s_rho: 31)
Dimensions without coordinates: one, s_rho
Data variables:
    tstart   (one) float64 ...
    tend     (one) float64 ...
    theta_s  (one) float64 ...
    theta_b  (one) float64 ...
    Tcline   (one) float64 ...
    hc       (one) float64 ...
    Cs_r     (s_rho) float64 ...
    sc_r     (s_rho) float64 ...

## Interpolate low resolution variables on high resolution grid.
### Interpolation function
Define function that takes low res data (as 2D or 3D data array), and high res grid and returns high resolution data array.

In [121]:
from scipy import interpolate

def low_to_high(lr_da,lr_grd,hr_grd,gt,dim,fill_value=0.0):
    
    print('set up empty hr data array')
    if dim == 2:
    
        dummy = np.zeros(hr_grd['lon_'+gt].shape)
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        hr_da = xr.DataArray(dummy,coords=[y,x],dims=['eta_'+gt,'xi_'+gt])
        
    elif dim == 3:
        
        N = lr_da.s_rho.sizehr_ini['temp'] = low_to_high(lr_rst.temp,lr_grd,hr_grd,'rho',3)hr_ini['temp'] = low_to_high(lr_rst.temp,lr_grd,hr_grd,'rho',3)
        dummy = np.tile(np.zeros(hr_grd['lon_'+gt].shape),(N,1,1))
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        z = lr_da['s_rho']
        hr_da = xr.DataArray(dummy,coords=[z,y,x],dims=['s_rho','eta_'+gt,'xi_'+gt])
    
    print('find index of shared first coordinate')
    # Find index of bottom left corner of low res data on high res grid
    ind = (hr_grd['x_'+gt].values == lr_grd['x_'+gt][0,0].values) & (hr_grd['y_'+gt].values == lr_grd['y_'+gt][0,0].values)
    eta0, xi0 = np.array(np.nonzero(ind)).squeeze()
    
    print('Fill in the mask of lr data')
    # Fill the mask of low resolution data with nearest neibghours and fill in known values on high res grid.
    if dim == 2:
        data = lr_da.values

        valid_mask = ~np.isnan(data)
        coords = np.array(np.nonzero(valid_mask)).T
        values = data[valid_mask]

        it = interpolate.NearestNDInterpolator(coords,values)

        filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
        
        # Fill in known values on high res grid
        hr_da[eta0::2,xi0::2] = filled
        
    if dim == 3:
        
        for k in np.arange(N):
            
            print('processing depth level: ',k)
            data = lr_da[k].values

            valid_mask = ~np.isnan(data)
            coords = np.array(np.nonzero(valid_mask)).T
            values = data[valid_mask]

            it = interpolate.NearestNDInterpolator(coords,values)

            filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
    
            # Fill in known values on high res grid
            hr_da[k,eta0::2,xi0::2] = filled
    
    # Now interpolate the intermediate grid points using x and y coordinates (in km from center point)
    print('interpolate at new intermediate cells on the hr grid')
    
    # Points we know the data
    x = hr_grd['x_'+gt][0,xi0::2].values
    y = hr_grd['y_'+gt][eta0::2,0].values
    
    # Define the target cells as meshgrid
    grid_x,grid_y = np.meshgrid(hr_grd['x_'+gt][0,:].values,hr_grd['y_'+gt][:,0].values)
    
    if dim == 2:
        # Data at these points
        values = hr_da[eta0::2,xi0::2].to_masked_array()

        # Define the interpolation function (fast regular grid interpolation)
        interp_func = interpolate.RegularGridInterpolator((y,x),values,bounds_error=False,fill_value=None)

        # Interpolate using linear interpolation
        interp = interp_func((grid_y[:,:],grid_x[:,:]))

        # Assign new data to data array
        hr_da[:,:] = interp
        
        # Fill with zeros where mask is present
        print('fill hr mask areas with fill value: ',fill_value)
        hr_da.values[hr_grd['mask_'+gt].values == 0] = fill_value
        
    if dim == 3:
        
        for k in np.arange(N):
            print('processing depth level: ',k)
            # Data at these points
            values = hr_da[k,eta0::2,xi0::2].to_masked_array()

            # Define the interpolation function (fast regular grid interpolation)
            interp_func = interpolate.RegularGridInterpolator((y,x),values,bounds_error=False,fill_value=None) 

            # Interpolate using linear interpolation
            interp = interp_func((grid_y[:,:],grid_x[:,:]))

            # Assign new data to data array
            hr_da[k,:,:] = interp

            # Fill with zeros where mask is present
            print('fill hr mask areas with fill value: ',fill_value)
            hr_da[k].values[hr_grd['mask_'+gt].values == 0] = fill_value
    
    return hr_da


### Function call for: zeta, ubar, vbar and u, v, temp, salt 
Get 2D and 3D high resolution data and assign to prepared ini file.

In [122]:
# interpolate 2D variables
hr_ini['zeta'] = low_to_high(lr_rst.zeta,lr_grd,hr_grd,'rho',2);
hr_ini['ubar'] = low_to_high(lr_rst.ubar,lr_grd,hr_grd,'u',2); 
hr_ini['vbar'] = low_to_high(lr_rst.vbar,lr_grd,hr_grd,'v',2);
# interpolate 3D variables
hr_ini['u'] = low_to_high(lr_rst.u,lr_grd,hr_grd,'u',3);
hr_ini['v'] = low_to_high(lr_rst.v,lr_grd,hr_grd,'v',3);
hr_ini['temp'] = low_to_high(lr_rst.temp,lr_grd,hr_grd,'rho',3)
hr_ini['salt
for var in ['zeta','ubar','vbar','u','v','temp','salt']:'] = low_to_high(lr_rst.salt,lr_grd,hr_grd,'rho',3)

set up empty hr data array
find index of shared first coordinate
Fill in the mask of lr data
interpolate at new intermediate cells on the hr grid
fill hr mask areas with fill value:  0.0
set up empty hr data array
find index of shared first coordinate
Fill in the mask of lr data
interpolate at new intermediate cells on the hr grid
fill hr mask areas with fill value:  0.0
set up empty hr data array
find index of shared first coordinate
Fill in the mask of lr data
interpolate at new intermediate cells on the hr grid
fill hr mask areas with fill value:  0.0
set up empty hr data array
find index of shared first coordinate
Fill in the mask of lr data
processing depth level:  0
processing depth level:  1
processing depth level:  2
processing depth level:  3
processing depth level:  4
processing depth level:  5
processing depth level:  6
processing depth level:  7
processing depth level:  8
processing depth level:  9
processing depth level:  10
processing depth level:  11
processing depth lev

fill hr mask areas with fill value:  0.0
processing depth level:  6
fill hr mask areas with fill value:  0.0
processing depth level:  7
fill hr mask areas with fill value:  0.0
processing depth level:  8
fill hr mask areas with fill value:  0.0
processing depth level:  9
fill hr mask areas with fill value:  0.0
processing depth level:  10
fill hr mask areas with fill value:  0.0
processing depth level:  11
fill hr mask areas with fill value:  0.0
processing depth level:  12
fill hr mask areas with fill value:  0.0
processing depth level:  13
fill hr mask areas with fill value:  0.0
processing depth level:  14
fill hr mask areas with fill value:  0.0
processing depth level:  15
fill hr mask areas with fill value:  0.0
processing depth level:  16
fill hr mask areas with fill value:  0.0
processing depth level:  17
fill hr mask areas with fill value:  0.0
processing depth level:  18
fill hr mask areas with fill value:  0.0
processing depth level:  19
fill hr mask areas with fill value:  0

Assign time dimension to new data arrays with the ocean_time set to the one from the low resolution solution.

In [149]:
hr_ini.coords['ocean_time'] = lr_rst.ocean_time
for var in ['zeta','ubar','vbar','u','v','temp','salt']:
    hr_ini[var] = hr_ini[var].expand_dims('ocean_time',0)

### Save new ini file
Compare high resolution and low resolution ini file for consistency and save as netcdf file making sure that ocean_time is saved as unlimited dimension.

In [152]:
lr_ini.zeta

<xarray.DataArray 'zeta' (ocean_time: 1, eta_rho: 730, xi_rho: 860)>
[627800 values with dtype=float64]
Coordinates:
  * ocean_time  (ocean_time) timedelta64[ns] 00:00:00
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:  free-surface
    units:      meter

In [153]:
hr_ini.zeta

<xarray.DataArray 'zeta' (ocean_time: 1, eta_rho: 1460, xi_rho: 1720)>
array([[[ 0.097331,  0.097331, ..., -0.125972, -0.126097],
        [ 0.097331,  0.097331, ..., -0.126097, -0.126097],
        ..., 
        [-0.519038, -0.518511, ...,  0.219246,  0.219093],
        [-0.518511, -0.518511, ...,  0.219093,  0.219093]]])
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 1999-12-31
  * eta_rho     (eta_rho) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * xi_rho      (xi_rho) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...

In [150]:
hr_ini_path =  os.path.join(os.pardir,'data','preprocessing','processed','waom5_ini.nc')
hr_ini.to_netcdf(hr_ini_path,unlimited_dims='ocean_time')